In [1]:
import pandas as pd
from urllib.parse import unquote
import tqdm
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import io
from numpy import dot
from numpy.linalg import norm
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import sys
import re
from datetime import datetime

2023-05-05 01:45:49.283809: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
df = pd.read_csv('../data/csic_database.csv')
data_df = pd.DataFrame()
data_df["classification"] = df['classification']
merge_url_content = lambda x : x['URL'].split()[0] + (('&' if "?" in x['URL'] else "?" + x['content']) if not pd.isna(x['content']) else '')
data_df["URL"]= df.apply( merge_url_content , axis = 1)

In [12]:
#V3

#some s+cript not being caught 49579



def multisplit(regex, string):
    return re.split(regex, string)

def get_delimiter(regex, string):
    return re.findall(regex, string)

def tokenize_url(x):
    x = unquote(x, encoding='cp1252') #45589
    x = unquote(x, encoding='cp1252') #double encoded url string (should probably have this in a loop)
    delimiter_string = "[?&=@\.%+<>()/~;'-*!#,:]|\r|\n" #could add _ but used in pswd like line 45584   "" are a problem 45728
    delimiters = get_delimiter(delimiter_string, x)
    tokens = multisplit(delimiter_string , x)
    new_url_tokens = []
    for index,token in enumerate(tokens):
        new_url_tokens.append(token.lower())
        if index < len(delimiters) :
            new_url_tokens.append(delimiters[index])
    new_url_tokens = [token for token in new_url_tokens if len(token)>0]  
    new_url_tokens = new_url_tokens + ['<pad>']*(309-len(new_url_tokens))
    return new_url_tokens

data_df["URL"]= data_df["URL"].apply( lambda x : tokenize_url(x) )


In [13]:
def generate_freq_dist(sequence_df, token_dict) :
    for token_list in sequence_df :
        if token_list is not None :
            for token in token_list : 
                token_dict[token] = (token_dict[token] if token in token_dict else 0 )+ 1
            
token_dict = {}
generate_freq_dist(data_df['URL'] ,token_dict)
token_dict = {k: v for k, v in sorted(token_dict.items(), key=lambda item: item[1], reverse = True)}
print(token_dict)

{'<pad>': 16843989, '/': 315474, '=': 202178, '&': 163056, ':': 126523, '+': 89955, '.': 77657, 'http': 61411, 'localhost': 61082, '8080': 61048, 'tienda1': 59066, 'jsp': 47523, '?': 40006, 'publico': 34763, 'b1': 23061, 'modo': 18468, 'registro': 14288, 'nombre': 14244, 'login': 14057, 'entrar': 13202, 'miembros': 12564, ',': 12503, "'": 11426, '@': 10425, 'imagenes': 9884, 'ciudad': 9569, 'registrar': 9437, 'provincia': 9427, 'password': 9421, 'apellidos': 9419, 'direccion': 9419, 'dni': 9417, 'email': 9415, 'cp': 9415, 'ntc': 9413, 'precio': 9075, 'carrito': 8952, 'id': 7908, 'vaciar': 7388, 'gif': 6340, '1': 5210, 'anadir': 4919, 'autenticar': 4860, 'editar': 4838, 'pagar': 4789, 'al': 4788, '2': 4737, 'añadir': 4727, 'cantidad': 4671, 'remember': 4615, 'pwd': 4613, 'insertar': 4526, 'global': 4417, 'jpg': 4273, 'de': 4260, '0': 4227, 'caracteristicas': 4039, 'credenciales': 3379, ';': 3206, '<': 3198, '>': 3198, 'incorrectas': 3169, '3': 2990, 'b2': 2982, 'errormsg': 2947, 'c': 28

In [14]:
vocab = {}
count = 0 
for item in token_dict:
    vocab[item] = count
    count = count + 1
    
inverse_vocab = {index: token for token, index in vocab.items()}

In [15]:
temp = list(data_df['URL'])
sequences = []
for temp_seq in temp:
    seq = []
    for token in temp_seq : 
        seq.append(vocab[token])
    sequences.append(seq)


In [16]:
# Converting from the vector embedding files to vector embeddings
vectors_df = pd.read_csv('vectors_v2.0.tsv', sep = '\t', header = None)

def generate_embedding(index):
    word_vector = data_df['URL'].iloc[index]
    embedded_vector = []
    for word in word_vector :  
        embedded_vector.append(vectors_df.iloc[vocab[word]])
    return np.array(embedded_vector)  

In [17]:
def get_avg_vector(index):
    vector_list = []
    for word in data_df['URL'].iloc[index]:
        if not word == '<pad>' :
            vector_list.append(list(vectors_df.iloc[vocab[word]]))
    return np.mean(vector_list, axis = 0)

In [18]:
clustering_df = pd.DataFrame(columns = ['classification', 'vectors'])
clustering_df['classification'] = data_df['classification']
clustering_df['vectors'] = [get_avg_vector(index) for index in range(len(data_df['URL']))]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(list(clustering_df['vectors']), list(clustering_df['classification']), train_size=0.8, random_state = 42)

#poly = svm.SVC(kernel='poly', degree=2, C=1, decision_function_shape='ovo').fit(X_train, y_train) 93%
#sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(X_train, y_train)

poly = svm.SVC(kernel='poly', degree=2, C=50000).fit(X_train, y_train)

poly_pred = poly.predict(X_test)
#sig_pred = sig.predict(X_test)


accuracy_poly = poly.score(X_test, y_test)
#accuracy_sig = sig.score(X_test, y_test)

print('Accuracy Polynomial Kernel:', accuracy_poly)
#print('Accuracy Sigmoid Kernel:”', accuracy_sig)

Accuracy Polynomial Kernel: 0.9948415622697127
